In [ ]:
from lsst_efd_client import EfdClient


In [ ]:
client = EfdClient('summit_efd')

## The EFD client annotate() method


In [ ]:
async def annotate(topic, field, start, timestamp="private_efdStamp", enum_class=None):
    """Annotate events in Chronograf.
    
    Parameters
    ----------
    topic : `str`
        Name of the EFD event topic.
    field : `str`
        Name of the field with the event to annotate.
    start: `str`
        Time to start querying events, e.g. "now()-1d"
    timestamp : `str`
        Name of the timestamp field to use for point annotations.
    enum_class : `enum` or None
        Enum class with event names, use that as the annotation text if provided.
    colors : `dict` or None
        Dictionary mapping event value to an hex color code
    
    Returns
    -------
    result : `bool`
        True if successfully write annotations to the chronograf database.
    """
    
    # Annotation start_time format is a unix time integer with ns precision
    events = await client.influx_client.query(f'''SELECT {field}, {timestamp}*1000000000 AS start_time FROM "{topic}" WHERE time > {start}''')

    if enum_class:
        # Use the event name from the enum class as the annotation text      
        events["text"] = events[field].map(lambda x: f'''{enum_class(x).name}''') 
    else:
        events["text"] = events[field]
    
    # Drop the initial field column 
    events = events.drop(columns=[field])
    
    # Add tags
    events['csc'] = topic.split(".")[2]
    events['state'] = events['text']
   
    # For point annotations, set the modified_time_ns to start_time
    events['modified_time_ns'] = events['start_time']

    # Annotations have a deleted flag, se to False by default
    events['deleted'] = False

    # Generate the annotation ID based on the timestmap
    events['id'] = events['start_time'].map(lambda x: hex(x))
    
    # Write annotations to Chronograf
    chronograf = EfdClient('summit_efd', db_name='chronograf')
    
    return await chronograf.influx_client.write(events, measurement="annotations", tag_columns=['csc', 'state', 'id'])
    

### lsst.sal.MTM1M3.logevent_detailedState events

In [ ]:
await client.get_schema("lsst.sal.MTM1M3.logevent_detailedState")

In [ ]:
from lsst.ts.idl.enums import MTM1M3

In [ ]:
await annotate(topic="lsst.sal.MTM1M3.logevent_detailedState", field="detailedState",  start="now()-1d", enum_class=MTM1M3.DetailedState)